In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Installing spark

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #downloading jdk

In [3]:
!wget -q http://apache.mirrors.pair.com/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz #downloading spark
!ls

drive  sample_data  spark-2.4.5-bin-hadoop2.7.tgz


In [0]:
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz # untar the downloaded spark file

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
df = spark.createDataFrame([{"Google": "Colab","Spark": "Scala"} ,{"Google": "Dataproc","Spark":"Python"}])
df.show()

/content/spark-2.4.5-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+--------+------+
|  Google| Spark|
+--------+------+
|   Colab| Scala|
|Dataproc|Python|
+--------+------+



## Assignment


In [0]:
data=spark.read.csv(
    "/content/drive/My Drive/Colab Notebooks/561/Amazon_Responded_Oct05.csv", header=True)

In [10]:
data.first()

Row(id_str="'793270689780203520'", tweet_created_at='Tue Nov 01 01:57:25 +0000 2016', user_screen_name='SeanEPanjab', user_id_str='143515471', user_statuses_count='51287', user_favourites_count='4079', user_protected='False', user_listed_count='74', user_following='False', user_description="Content marketer; Polyglot; Beard aficionado; Sikh. Persian, Catalan, French, Spanish. You'll find lol in the interstitial lulls of my tweets.", user_location='غریب الوطن', user_verified='False', user_followers_count='1503', user_friends_count='850', user_created_at='Thu May 13 17:43:52 +0000 2010', tweet_language='en', text_='@AmazonHelp Can you please DM me? A product I ordered last year never arrived.', favorite_count='0', favorited='False', in_reply_to_screen_name='AmazonHelp', in_reply_to_status_id_str=None, in_reply_to_user_id_str='85741735', retweet_count='0', retweeted='False', text=None)

In [11]:
data.withColumnRenamed

<bound method DataFrame.withColumnRenamed of DataFrame[id_str: string, tweet_created_at: string, user_screen_name: string, user_id_str: string, user_statuses_count: string, user_favourites_count: string, user_protected: string, user_listed_count: string, user_following: string, user_description: string, user_location: string, user_verified: string, user_followers_count: string, user_friends_count: string, user_created_at: string, tweet_language: string, text_: string, favorite_count: string, favorited: string, in_reply_to_screen_name: string, in_reply_to_status_id_str: string, in_reply_to_user_id_str: string, retweet_count: string, retweeted: string, text: string]>

In [0]:
x=data.select("id_str","tweet_created_at","user_verified","favorite_count","retweet_count","text_")

In [13]:
x.withColumnRenamed
x.show()


+--------------------+--------------------+-------------+--------------+-------------+--------------------+
|              id_str|    tweet_created_at|user_verified|favorite_count|retweet_count|               text_|
+--------------------+--------------------+-------------+--------------+-------------+--------------------+
|'793270689780203520'|Tue Nov 01 01:57:...|        False|             0|            0|@AmazonHelp Can y...|
|'793281386912354304'|Tue Nov 01 02:39:...|         True|             0|            0|@SeanEPanjab I'm ...|
|'793501578766319616'|Tue Nov 01 17:14:...|        False|             0|            0|@AmazonHelp It wa...|
|'793501657346682880'|Tue Nov 01 17:15:...|        False|             0|            0|@AmazonHelp I am ...|
|'793502854459879424'|Tue Nov 01 17:19:...|         True|             0|            0|@SeanEPanjab Plea...|
|'793504235400884224'|Tue Nov 01 17:25:...|         True|             0|            0|@SeanEPanjab With...|
|'793511847899070465'|Tue No

# Task 1 

Step 1- filtering records for verified users

In [18]:
data_task1=x.filter(x.user_verified=="True")
data_task1.count()
#data_task1.show()

171797

Step 2-converting the date format and finding the day with highest number of tweets

In [0]:
data_step2 = data_task1.toPandas()
data_step2['tweet_created_at'] = data_step2.tweet_created_at.str.slice(4,10)+' '+data_step2.tweet_created_at.str.slice(26,30)

In [20]:
#converting the datatype to spark dataframe and finding max tweets on which date
data_step2_out=spark.createDataFrame(data_step2)
date=data_step2_out.groupBy(data_step2_out["tweet_created_at"]).count()
highest_tweets=date.orderBy(date["count"].desc()).show(1)

+----------------+-----+
|tweet_created_at|count|
+----------------+-----+
|     Jan 03 2017| 1536|
+----------------+-----+
only showing top 1 row




Step 3--Find word frequency for top 100 tweets of the day which had highest no. of tweets

In [34]:
#top 100 tweets from the day with highest tweets
data_step2_out.createOrReplaceTempView("table1")
df = spark.sql("SELECT text_, favorite_count+retweet_count as sum FROM table1  where tweet_created_at='Jan 03 2017' order by sum desc limit 100 ")
df.show(10)
top_100_tweets=df.toPandas()
top_100_tweets.loc[0:9,:]

+--------------------+---+
|               text_|sum|
+--------------------+---+
|@amazon worst sho...|5.0|
|@ItsJosshA We alw...|3.0|
|@ItsJosshA Oh no!...|2.0|
|@KStefl Sounds li...|2.0|
|@Schoey1992 Happy...|2.0|
|@ratbones666 You ...|2.0|
|@ThorpPerrow Awww...|2.0|
|@thedexterouz Hi!...|2.0|
|@matt_linsley Ple...|1.0|
|@VlSlT Sorry to h...|1.0|
+--------------------+---+
only showing top 10 rows



,text_,sum
0,"@amazon worst shopping experience, no servic...",5.0
1,@ItsJosshA We always aim to deliver by the dat...,3.0
2,@ratbones666 You so fancy!!! You already knooo...,2.0
3,"@ThorpPerrow Awww! Happy Birthday, you don't l...",2.0
4,@KStefl Sounds like you know what to add to yo...,2.0
5,@thedexterouz Hi! We'd like to help. When you ...,2.0
6,"@Schoey1992 Happy Birthday, Matt! #FriendsFore...",2.0
7,@ItsJosshA Oh no! I'm sorry! Please reach out ...,2.0
8,"@VlSlT Sorry to hear that, contact us here: ht...",1.0
9,@matt_linsley Please keep us posted on this an...,1.0


In [0]:
## To find the word frequency
word_freq= top_100_tweets["text_"]
word_freq=word_freq.tolist()



In [36]:
import nltk
import pandas as pd
import string
import re
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# Data cleaning for finding word frequency
def removal(textfile):
    y=textfile
    cleaned=[]
    for i in y:
      # print(i)
      i.replace("'", " ")  #step 1: replacing apostraphe
      i=re.sub(r"\@\w+"," ",i)   #step2: removing words starting with @
      cleaned.append(i.translate(str.maketrans(string.punctuation,' '*len(string.punctuation)))) # step 3: remove_punctuation               
    
    output=[]
    for i in cleaned:
        output.append(" ".join([w.lower() for w in i.split()  if w.isalpha()]))    

    return output

In [0]:

cleaned_list=removal(word_freq)

In [0]:
#Using data frame and for loop
li = []
j = 0
wordfreq = {}
for i in range(0,len(cleaned_list)):
    li.append(nltk.word_tokenize(cleaned_list[i]))
for j in range(len(li)):
    tokens = li[j]
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

In [0]:
wordfreq=pd.DataFrame(wordfreq.items(),columns=["word","frequency"])

In [0]:
wordfreq.to_csv(r"/content/drive/My Drive/Colab Notebooks/561/word_frequency.csv")

# Using RDD to find  word frequency

In [64]:
#using RDD to find  word frequency
from pyspark import SparkContext
sc =SparkContext.getOrCreate()
k=cleaned_list
tuple(k)

#%%

rdd = sc.parallelize(k)
word_frequency_rdd = rdd.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()
print(word_frequency_rdd)
print(len(word_frequency_rdd))
#%%

tweet_word_frequency_rdd=pd.DataFrame(word_frequency_rdd)
tweet_word_frequency_rdd.to_csv(r"/content/drive/My Drive/Colab Notebooks/561/tweet_word_frequency_rdd.csv")


[('worst', 1), ('no', 5), ('service', 1), ('substantial', 1), ('delivery', 6), ('week', 1), ('we', 46), ('always', 3), ('in', 13), ('confirmation', 1), ('have', 18), ('update', 2), ('tracking', 5), ('fancy', 1), ('already', 1), ('don', 5), ('look', 7), ('pass', 2), ('details', 6), ('https', 44), ('co', 45), ('lil', 1), ('sounds', 1), ('like', 7), ('know', 13), ('playlist', 1), ('this', 27), ('year', 2), ('bv', 2), ('d', 7), ('help', 9), ('when', 5), ('connect', 1), ('us', 28), ('matt', 1), ('friendshipgoals', 1), ('oh', 1), ('i', 23), ('out', 6), ('into', 6), ('options', 3), ('hear', 4), ('as', 4), ('let', 7), ('doesn', 5), ('arrive', 6), ('tomorrow', 2), ('an', 8), ('ar', 3), ('apologies', 2), ('incorrect', 1), ('item', 5), ('using', 2), ('above', 2), ('do', 4), ('further', 5), ('concerns', 1), ('ca', 1), ('thanks', 11), ('shout', 1), ('looking', 2), ('improve', 1), ('share', 1), ('specific', 1), ('feedback', 7), ('via', 4), ('form', 1), ('sj', 6), ('delay', 1), ('indicate', 1), ('new

## Task 2

In [0]:
find_text=spark.read.csv(
    "/content/drive/My Drive/Colab Notebooks/561/find_text.csv", header=True)
amazon_data=spark.read.csv(
    "/content/drive/My Drive/Colab Notebooks/561/Amazon_Responded_Oct05.csv", header=True)

In [0]:
amazon_data.createOrReplaceTempView("amazon")
find_text.createOrReplaceTempView("findtext")
find_text_updated = spark.sql("SELECT f.id_str,a.text_ as text FROM amazon a join findtext f on a.id_str= f.id_str")
find_text_updated=find_text_updated.toPandas()
find_text_updated.to_csv(r'/content/drive/My Drive/Colab Notebooks/561/Findtext_updated2.csv')